In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv


In [2]:
import torch
from sklearn.preprocessing import MinMaxScaler ,LabelEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Preprocessed data

In [3]:
df = pd.read_csv('/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv',index_col = 'loan_id')
df

no_of_dependents      education  self_employed   income_annum  \
loan_id                                                                   
1                        2       Graduate             No        9600000   
2                        0   Not Graduate            Yes        4100000   
3                        3       Graduate             No        9100000   
4                        3       Graduate             No        8200000   
5                        5   Not Graduate            Yes        9800000   
...                    ...            ...            ...            ...   
4265                     5       Graduate            Yes        1000000   
4266                     0   Not Graduate            Yes        3300000   
4267                     2   Not Graduate             No        6500000   
4268                     1   Not Graduate             No        4100000   
4269                     1       Graduate             No        9200000   

          loan_amount   loan_term   cibil_score   residential_assets_value  \
loan_id                                                                      
1            29900000          12           778                    2400000   
2            12200000           8           417                    2700000   
3            29700000          20           506                    7100000   
4            30700000           8           467                   18200000   
5            24200000          20           382                   12400000   
...               ...         ...           ...                        ...   
4265          2300000          12           317                    2800000   
4266         11300000          20           559                    4200000   
4267         23900000          18           457                    1200000   
4268         12800000           8           780                    8200000   
4269         29700000          10           607                   17800000   

          commercial_assets_value   luxury_assets_value   bank_asset_value  \
loan_id                                                                      
1                        17600000              22700000            8000000   
2                         2200000               8800000            3300000   
3                         4500000              33300000           12800000   
4                         3300000              23300000            7900000   
5                         8200000              29400000            5000000   
...                           ...                   ...                ...   
4265                       500000               3300000             800000   
4266                      2900000              11000000            1900000   
4267                     12400000              18100000            7300000   
4268                       700000              14100000            5800000   
4269                     11800000              35700000           12000000   

         loan_status  
loan_id               
1           Approved  
2           Rejected  
3           Rejected  
4           Rejected  
5           Rejected  
...              ...  
4265        Rejected  
4266        Approved  
4267        Rejected  
4268        Approved  
4269        Approved  

[4269 rows x 12 columns]

In [4]:
df[df.isna() == True].count()

 no_of_dependents            0
 education                   0
 self_employed               0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
 loan_status                 0
dtype: int64

# Note here there is a problem in dataset column as there is space in names which cause problem so let us rename the columns.

In [5]:
df.rename(columns = {' no_of_dependents': 'no_of_dependents',
                     ' education' : 'education', 
                     ' self_employed' : 'self_employed',
                     ' income_annum' : 'income_annum',
                     ' loan_amount' : 'loan_amount',
                     ' loan_term' :'loan_term',
                     ' cibil_score' : 'cibil_score',
                     ' residential_assets_value' : 'residential_assets_value', 
                     ' commercial_assets_value' : 'commercial_assets_value',
                     ' luxury_assets_value' : 'luxury_assets_value',
                     ' bank_asset_value' : 'bank_asset_value',
                     ' loan_status' : 'loan_status'}, 
                      inplace = True)
df.columns
col = ['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term',
       'cibil_score', 'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'education_ Graduate',
       'education_ Not Graduate', 'self_employed_ No', 'self_employed_ Yes',
       'Total_assets_value']
Scaling = ['residential_assets_value','commercial_assets_value',	'luxury_assets_value'	,'bank_asset_value']

In [6]:
label_encoder  =LabelEncoder()
mx = MinMaxScaler()
def preprocess(df):
    y = df.pop('loan_status')
    y = label_encoder.fit_transform(y)
    
    df = pd.get_dummies(df)
    
    df['Total_assets_value'] = 0
    for i in Scaling:
        df['Total_assets_value'] += df[i] 
    
    df[col]=mx.fit_transform(df[col])
    df = df.drop(columns = Scaling)
    
    
    
    return df,y

df,y = preprocess(df)

## Graphing

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Comparision')
sns.barplot( ax=axes[0, 0] ,x=y, y=df['Total_assets_value'])
sns.barplot( ax=axes[0, 1] ,x=y, y=df['income_annum'])
sns.barplot( ax=axes[0, 2] ,x=y, y=df['loan_amount'])
sns.barplot( ax=axes[1, 0] ,x=y, y=df['loan_term'])
sns.barplot( ax=axes[1, 1] ,x=y, y=df['cibil_score'])
sns.barplot( ax=axes[1, 2] ,x=y, y=df['no_of_dependents'])

<Axes: ylabel='no_of_dependents'>

# Modeling

In [9]:
import torch
torch.__version__
from sklearn.model_selection import train_test_split

In [10]:
len(df.columns)

10

In [11]:
X_train, X_test,y_train, y_test = train_test_split(df,y ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

In [12]:
 df.values

array([[0.4       , 0.96907216, 0.75510204, ..., 1.        , 0.        ,
        0.55703212],
       [0.        , 0.40206186, 0.30357143, ..., 0.        , 1.        ,
        0.18383167],
       [0.6       , 0.91752577, 0.75      , ..., 1.        , 0.        ,
        0.6345515 ],
       ...,
       [0.4       , 0.64948454, 0.60204082, ..., 1.        , 0.        ,
        0.42746401],
       [0.2       , 0.40206186, 0.31887755, ..., 1.        , 0.        ,
        0.3145072 ],
       [0.2       , 0.92783505, 0.75      , ..., 1.        , 0.        ,
        0.85160576]])

In [13]:

X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [14]:
import torch
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [15]:
# One way to create
class loaner(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Linear(in_features = 10 , out_features= 5 )
        self.layer2 = nn.Linear(in_features =  5 , out_features= 1 )
        
    def forward(self,x):
         return  self.layer2(self.layer1(x))

model = loaner().to(device)
model

loaner(
  (layer1): Linear(in_features=10, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=1, bias=True)
)

In [16]:
model = nn.Sequential(
    nn.Linear(in_features = 10, out_features =10),
    nn.Linear(in_features = 10, out_features =5),
    nn.Linear(in_features = 5, out_features =1)).to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params = model.parameters(),
                           lr = 0.7)
model

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=5, bias=True)
  (2): Linear(in_features=5, out_features=1, bias=True)
)

In [17]:
def accuracy_fn(y_true,y_preds):
    correct = torch.eq(y_true,y_preds).sum().item()
    acc = (correct/len(y_preds) * 100)
    return acc

## preparing training and testing loop

In [18]:
x_train , y_train = X_train.to(device) , y_train.to(device)
x_test , y_test = X_test.to(device) , y_test.to(device)

In [19]:
epoch = 100



for epochs in range(epoch):
    model.train()
    
    #forwaed pass
    y_logits = model(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    
    
    #calculate loss
    loss = loss_fn(y_logits,y_train)
    
    acc = accuracy_fn(y_true = y_train,
                      y_preds = y_pred)
    
    #optimizer
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    model.eval()
    with torch.inference_mode():
        test_logits = model(X_test).squeeze()
        test_preds = torch.round(torch.sigmoid(test_logits))
        
        test_loss = loss_fn(test_logits,
                           y_test)
        test_acc = accuracy_fn(y_true = y_test,
                              y_preds = test_preds)
    
    if epochs % 10 == 0:
         print(f"Epochs :{epochs} | loss: {loss:.{5}f} | acc: {acc:.{5}f}% | test_loss: {test_loss:.{5}f} | test_acc: {test_acc:.{5}f}% ")

Epochs :0 | loss: 0.66427 | acc: 62.35551% | test_loss: 0.66361 | test_acc: 61.79775% 
Epochs :10 | loss: 0.64021 | acc: 62.35551% | test_loss: 0.63739 | test_acc: 61.79775% 
Epochs :20 | loss: 0.55485 | acc: 67.72883% | test_loss: 0.53715 | test_acc: 69.75655% 
Epochs :30 | loss: 0.31422 | acc: 90.56545% | test_loss: 0.30339 | test_acc: 90.44944% 
Epochs :40 | loss: 0.63742 | acc: 54.98282% | test_loss: 0.56224 | test_acc: 86.89139% 
Epochs :50 | loss: 0.27232 | acc: 91.50266% | test_loss: 0.26976 | test_acc: 90.26217% 
Epochs :60 | loss: 0.23252 | acc: 90.97157% | test_loss: 0.25472 | test_acc: 91.10487% 
Epochs :70 | loss: 0.26767 | acc: 91.84630% | test_loss: 0.26530 | test_acc: 90.63670% 
Epochs :80 | loss: 0.22724 | acc: 92.18994% | test_loss: 0.23797 | test_acc: 90.82397% 
Epochs :90 | loss: 0.22300 | acc: 92.22118% | test_loss: 0.23566 | test_acc: 90.54307% 


# Other model trainings

In [20]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Generate sample data
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the classification models
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier(n_estimators=100)),
    ('Gradient Boosting', GradientBoostingClassifier(n_estimators=100)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('Support Vector Machine', SVC()),
    ('Neural Network', MLPClassifier()),
    ('XGBoost', XGBClassifier()),
    ('LightGBM', LGBMClassifier()),
    ('CatBoost', CatBoostClassifier(verbose=0)),
    ('Extra Trees', ExtraTreesClassifier())
]

# Train and evaluate each model
results = []
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results.append((name, accuracy))

# Display the results
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy'])
print(results_df)


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                     Model  Accuracy
0      Logistic Regression     0.830
1            Decision Tree     0.835
2            Random Forest     0.875
3        Gradient Boosting     0.900
4      K-Nearest Neighbors     0.800
5              Naive Bayes     0.810
6   Support Vector Machine     0.830
7           Neural Network     0.830
8                  XGBoost     0.880
9                 LightGBM     0.880
10                CatBoost     0.875
11             Extra Trees     0.870
